In [4]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import display, Markdown
from datetime import datetime

load_dotenv(override=True)

True

In [ ]:
params = {"command": "npx", "args": ["cv-forge"]}


In [15]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools[14]

Tool(name='save_cover_letter_pdf', title=None, description='Generate and save a cover letter as PDF to specified location', inputSchema={'type': 'object', 'properties': {'userProfile': {'type': 'object', 'properties': {'personalInfo': {'type': 'object', 'properties': {'fullName': {'type': 'string'}, 'email': {'type': 'string'}, 'phone': {'type': 'string'}, 'location': {'type': 'string'}, 'linkedIn': {'type': 'string'}, 'github': {'type': 'string'}, 'website': {'type': 'string'}}, 'required': ['fullName', 'email'], 'additionalProperties': False}, 'summary': {'type': 'string', 'description': 'Professional summary'}, 'experience': {'type': 'array', 'items': {'type': 'object', 'properties': {'jobTitle': {'type': 'string'}, 'company': {'type': 'string'}, 'location': {'type': 'string'}, 'startDate': {'type': 'string'}, 'endDate': {'type': 'string'}, 'description': {'type': 'string'}, 'achievements': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['jobTitle', 'company', 'startDa

In [12]:
from pypdf import PdfReader

reader = PdfReader("resume.pdf")

resume_text = ""
for page in reader.pages:
    resume_text += page.extract_text()

job_description = """React Developer

Choosing Capgemini means choosing a company where you will be empowered to shape your career in the way you'd like, where you'll be supported and inspired by a collaborative community of colleagues around the world, and where you'll be able to reimagine what's possible. Join us and help the world's leading organizations unlock the value of technology and build a more sustainable, more inclusive world.
Role Overview
As a React Developer you will be responsible for building and maintaining responsive web applications using React within the Financial Services Insurance domain You will work closely with UIUX designers backend developers and business teams to deliver high-quality scalable and secure solutions that meet client requirements
Key Responsibilities
Develop and maintain web applications using React and related technologies
Implement reusable components and ensure responsive design across devices
Collaborate with backend teams to integrate APIs and services
Optimize application performance and ensure cross browser compatibility
Participate in code reviews and maintain coding standards
Work in Agile teams and contribute to sprint planning and delivery
Ensure compliance with security and quality standards
Required Skills and Experience
Strong proficiency in React and JavaScript ES6
Experience with state management libraries such as Redux or Context API
Knowledge of HTML5 CSS3 and responsive design principles
Familiarity with RESTful APIs and JSON data handling
Understanding of Agile methodology and SDLC practices
Good problem-solving and debugging skills
Exposure to TypeScript and modern React features such as Hooks
Knowledge of testing frameworks like Jest or React Testing Library
Experience with CICD tools and version control systems like Git
Understanding of insurance domain processes and workflows"""

signature = "Akash Hadagali Persett"


In [18]:
instructions = f"""
You are a professional cover letter writer and tool-using agent.

CRITICAL TOOLING RULES:
- The resume text is provided directly below. Do NOT try to read any files.
- You MUST use cv-forge tools to generate the cover letter text.
- Do NOT attempt to save files. Only generate the cover letter content.

RESUME TEXT:
{resume_text}

JOB DESCRIPTION:
{job_description}

SIGNATURE:
- The ending signature name MUST be: {signature}

TASK STEPS (execute in order):
1) Build a cv-forge-compatible JSON object userProfile from the resume text above with these fields:
   - personalInfo.fullName, personalInfo.email (required)
   - summary, experience[], education[], skills.technical[] (required)
   - projects[] (optional)
   - If you are describing the Sentinel project, mention "Agentic AI" in the description.
2) Build jobRequirements using the cv-forge parse_job_requirements tool on the job description.
   Ensure it has jobTitle, company, and jobDescription fields.
3) Call cv-forge generate_cover_letter with userProfile and jobRequirements.
4) Polish the output:
   - Personalised, specific, human — not templated or AI-sounding.
   - Professional tone. No clichés ("passionate", "team player").
   - No em-dashes (—). Use a regular hyphen (-) if needed.
   - Do not ask questions at the end.
   - If the JD lists multiple roles, write ONE letter that suits all of them.
5) Output ONLY the cover letter text — no headings, no markdown, no extra commentary.
6) Also output the company name at the end from the job description.
"""

In [25]:
from pydantic import BaseModel, Field

class CoverLetter(BaseModel):
    cover_letter: str = Field(description="The cover letter output")
    company_name: str = Field(description="The company name from the job description")



In [ ]:
request = "Generate a cover letter for the job description and resume text above."

model = "gpt-5.2"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="Cover Letter Writer", instructions=instructions, model=model, mcp_servers=[mcp_server], output_type=CoverLetter)
    with trace("cover-letter-writer"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output.cover_letter))
    display(Markdown(result.final_output.company_name))

RunResult:
- Last agent: Agent(name="Cover Letter Writer", ...)
- Final output (CoverLetter):
    {
      "cover_letter": "Akash Hadagali Persetti\nBloomington, IN 47404\n812-837-7651 | hadagalipersettiakash@gmail.com\nlinkedin.com/in/akash-hp\n\nFebruary 24, 2026\n\nHiring Manager\nCapgemini\n\nRe: React Developer\n\nDear Hiring Manager,\n\nI am applying for the React Developer role at Capgemini. I am an M.S. Computer Science candidate at Indiana University Bloomington (GPA 3.8) with hands-on experience building responsive, performance-focused web experiences and full-stack applications using React and JavaScript, plus a strong foundation in software engineering, databases, and cloud.\n\nIn my independent full-stack project, Sculptica, I built a React and Node.js web application backed by MySQL, implementing secure authentication, role-based access control, and core workflow automation. That end-to-end ownership required designing reusable UI flows, handling JSON-based API interaction

In [29]:
print(result.final_output.company_name)

Capgemini


In [12]:
import anthropic

message = anthropic.Anthropic().messages.create(
    model="claude-sonnet-4-0",
    max_tokens=1024,
    messages=[{"role": "user", "content": "Hello, Claude"}],
)

print(message.content[0].text)

Hello! It's nice to meet you. How are you doing today? Is there anything I can help you with or would you like to chat about?


In [15]:
from openai import OpenAI
gemini = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

response = gemini.chat.completions.create(
    model="gemini-3-flash-preview",
    messages=[{"role": "user", "content": "Hello, Google"}],
)

print(response.choices[0].message.content)


Hello! How can I help you today?
